In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from matplotlib.image import imread
import seaborn as sns
import os
import zipfile
import matplotlib.image as mpimg
import seaborn as sns
import random
import os
from zipfile import ZipFile
from skimage.io import imread, imshow
import warnings
warnings.filterwarnings('ignore')

In [ ]:
print(os.listdir("../input/dogs-vs-cats"))

In [ ]:
trainDataPath = "/kaggle/input/dogs-vs-cats/train.zip"
testDataPath = "/kaggle/input/dogs-vs-cats/test1.zip"
ZipFile(trainDataPath,mode = "r").extractall()
ZipFile(testDataPath,mode =  "r").extractall()

In [ ]:
train_file= './train'
test_file = './test1'

In [ ]:
img = load_img(train_file +  '/cat.0.jpg')
plt.imshow(img)

In [ ]:
train_list = os.listdir(train_file)
test_list = os.listdir(test_file)

In [ ]:
print (train_list[:5])
print (test_list[:5])

In [ ]:
print ('Toplam eğitim veri sayısı:',len(train_list))
print ('Toplam test veri sayısı:',len(test_list))

In [ ]:
categories = []
for filename in train_list:
    category = filename.split('.')[0]
    if category == 'dog':
        categories.append(1)
    else:
        categories.append(0)

data_frame = pd.DataFrame({ 'filename': train_list, 'category': categories})

In [ ]:
data_frame[:15]

In [ ]:
print (categories[:15])

In [ ]:
print (train_list[:15])

In [ ]:
data_frame["category"] = data_frame["category"].replace({0: 'cat', 1: 'dog'}) 

In [ ]:
x_train, val_set = train_test_split(data_frame, test_size=0.20, random_state=42)
x_train = x_train.reset_index(drop=True)
val_set = val_set.reset_index(drop=True)

In [ ]:
print ('Toplam eğitim veri sayısı:',len(x_train))
print ('Toplam val veri sayısı:',len(val_set))

In [ ]:
train_class_counts=x_train['category'].value_counts()
print("Train data setimizdeki cat sayısı:{}".format(train_class_counts[0]))
print("Train data setimizdeki dog sayısı:{}".format(train_class_counts[1]))

val_class_counts=val_set['category'].value_counts()
print("Val data setimizdeki cat sayısı:{}".format(val_class_counts[0]))
print("Val data setimizdeki dog sayısı:{}".format(val_class_counts[1]))

In [ ]:
train_rescale=ImageDataGenerator(rescale=1./255)

In [ ]:
train_model=train_rescale.flow_from_dataframe(   x_train,
                                                 directory = train_file, 
                                                 x_col='filename',
                                                 y_col='category',
                                                 color_size='rgb',
                                                 target_size=(128,128),
                                                 class_mode='categorical',
                                                 batch_size=64               )

In [ ]:
val_rescale=ImageDataGenerator(rescale=1./255)

In [ ]:
val_model=val_rescale.flow_from_dataframe(       val_set,
                                                 directory = train_file, 
                                                 x_col='filename',
                                                 y_col='category',
                                                 color_size='rgb',
                                                 target_size=(128,128),
                                                 class_mode='categorical',
                                                 batch_size=64                            )

In [ ]:
model_17=Sequential()

In [ ]:
model_17.add(Conv2D(128, kernel_size=(3, 3), activation='relu', input_shape=(128,128,3)))
model_17.add(MaxPooling2D(pool_size=(2,2)))
model_17.add(Conv2D(64,kernel_size=(3, 3), activation='relu'))
model_17.add(MaxPooling2D(pool_size=(2,2)))
model_17.add(Dropout(0.25))
model_17.add(Conv2D(32,kernel_size=(3,3),activation='relu'))
model_17.add(MaxPooling2D(pool_size=(2,2)))
model_17.add(Dropout(0.25))
model_17.add(Flatten())
model_17.add(Dense(128, activation='relu'))
model_17.add(Dropout(0.5))
model_17.add(Dense(2, activation='sigmoid'))

In [ ]:
model_17.summary()

In [ ]:
model_17.compile(loss='categorical_crossentropy', optimizer='RMSprop', metrics=['accuracy'])

In [ ]:
model_17.optimizer.lr=0.001

In [ ]:
callback= EarlyStopping(monitor='val_loss',patience=14, restore_best_weights=True, verbose=1, mode='min')

In [ ]:
batch_size=64
epochs=60

In [ ]:
history_17= model_17.fit(train_model,
          batch_size=batch_size,
          epochs=epochs,
          shuffle=True,
          verbose=1,
          callbacks= callback, 
          validation_data=val_model
                      )

In [ ]:
plt.plot(history_17.history['loss'])
plt.plot(history_17.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['loss', 'val'], loc='upper right')
plt.show()

In [ ]:
plt.plot(history_17.history['accuracy'])
plt.plot(history_17.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['acc', 'val',], loc='lower right')
plt.show()

In [ ]:
trainaug=ImageDataGenerator(
    rotation_range=15,
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
)

In [ ]:
train_newset=trainaug.flow_from_dataframe(
    x_train, 
    directory = train_file, 
    x_col='filename',
    y_col='category',
    target_size=(128,128),
    class_mode='categorical',
    batch_size=batch_size
)

In [ ]:
new_val=ImageDataGenerator(rescale=1./255)

In [ ]:
val_newset=new_val.flow_from_dataframe(          val_set,
                                                 directory = train_file, 
                                                 x_col='filename',
                                                 y_col='category',
                                                 color_size='rgb',
                                                 target_size=(128,128),
                                                 class_mode='categorical',
                                                 batch_size=64                        )

In [ ]:
example_df = x_train.sample(n=1).reset_index(drop=True)
example_generator = trainaug.flow_from_dataframe(   
    example_df,   
    './train/', 
    x_col='filename',
    y_col='category',
    target_size=(128,128),
    class_mode='categorical'
)

In [ ]:
plt.figure(figsize=(12, 12))
for i in range(0, 15):
    plt.subplot(5, 3, i+1)
    for X_batch, Y_batch in example_generator:
        image = X_batch[0]
        plt.imshow(image)
        break
plt.tight_layout()
plt.show()

In [ ]:
model_19=Sequential()

In [ ]:
model_19.add(Conv2D(128, kernel_size=(3, 3), activation='relu', input_shape=(128,128,3)))
model_19.add(MaxPooling2D(pool_size=(2,2)))
model_19.add(Dropout(0.25))
model_19.add(Conv2D(64,kernel_size=(3, 3), activation='relu'))
model_19.add(MaxPooling2D(pool_size=(2,2)))
model_19.add(Dropout(0.25))
model_19.add(Conv2D(32,kernel_size=(3,3),activation='relu'))
model_19.add(MaxPooling2D(pool_size=(2,2)))
model_19.add(Dropout(0.25))
model_19.add(Flatten())
model_19.add(Dense(128, activation='relu'))
model_19.add(Dropout(0.3))
model_19.add(Dense(128, activation='relu'))
model_19.add(Dropout(0.5))
model_19.add(Dense(2, activation='softmax'))

In [ ]:
model_19.summary()

In [ ]:
model_19.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model_19.optimizer.lr=0.001

In [ ]:
callback_2= EarlyStopping(monitor='val_loss',patience=14, restore_best_weights=True, verbose=1, mode='min')

In [ ]:
history_19= model_19.fit(train_newset,
                batch_size=batch_size,
                epochs=epochs,
                shuffle=True,
                verbose=1,
                callbacks= callback_2, 
                validation_data=val_newset
                                       )

In [ ]:
plt.plot(history_19.history['accuracy'])
plt.plot(history_19.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['acc', 'val',], loc='lower right')
plt.show()

In [ ]:
plt.plot(history_19.history['loss'])
plt.plot(history_19.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['loss', 'val'], loc='upper right')
plt.show()

In [ ]:
# test_file = './test1'
# test_list = os.listdir(test_file)


test_data_files= pd.DataFrame({
    
    'filename': test_list
    
})


In [ ]:
nb_samples =test_data_files.shape[0]

In [ ]:
generate_test_data_files= ImageDataGenerator(rescale=1./255)
test_gen = generate_test_data_files.flow_from_dataframe(
    test_data_files, 
    './test1' , 
    x_col='filename',
    #y_col=None,
    color_size='rgb',
    class_mode=None,
    target_size=(128,128),
    batch_size=64,
    shuffle=False
)

In [ ]:
predict = model_19.predict_generator(test_gen, steps=np.ceil(nb_samples/batch_size))

In [ ]:
test_data_files['category'] = np.argmax(predict, axis=-1)

In [ ]:
label_map = dict((v,k) for k,v in train_newset.class_indices.items())
test_data_files['category'] = test_data_files['category'].replace(label_map)

In [ ]:
test_data_files['category'] = test_data_files['category'].replace({ 'dog': 1, 'cat': 0 })

In [ ]:
submission_df = test_data_files.copy()
submission_df['fn'] = submission_df['filename'].str.split('.').str[0]
submission_df['label'] = submission_df['category']



In [ ]:
submission_df.drop(['filename', 'category'], axis=1, inplace=True)

In [ ]:
print (submission_df)

In [ ]:
data_frame_sub= pd.DataFrame({
    
    'fn': test_list,
    'label' : test_data_files['category']
    
})

In [ ]:
data_frame_sub.head(10000)

In [ ]:
img_sub = load_img(test_file +  '/7038.jpg')
plt.imshow(img_sub)

In [ ]:
img_test = data_frame_sub.head(25)
img_test.head(25)

In [ ]:
plt.figure(figsize=(15,15))

for index, row in img_test.iterrows():
    fn = row['fn']
    label = row['label']
    test_img= load_img('./test1/'+ fn, target_size=(128,128))
    plt.subplot(5,5,index+1)
    plt.imshow(test_img)
    plt.xlabel(fn + '(' + "{}".format(label) + ')' )
    
plt.tight_layout()
plt.show()    
    
 
